## 地点推荐系统

## 读取数据

### 读取位置信息 

In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
posdatasrc = './data/mappedpoiinfo.txt'
pos_dataset = pd.read_table(posdatasrc, header = None, delim_whitespace = True, names = ['ID', 'latitude', 'longtitude', 'city', 'subclass1', 'subclass2'])

### 查看位置信息

In [4]:
pos_dataset.info()
print('Columns with null values:\n', pos_dataset.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151106 entries, 0 to 151105
Data columns (total 6 columns):
ID            151106 non-null int64
latitude      151106 non-null float64
longtitude    151106 non-null float64
city          148472 non-null object
subclass1     90632 non-null object
subclass2     90632 non-null object
dtypes: float64(2), int64(1), object(3)
memory usage: 6.9+ MB
Columns with null values:
 ID                0
latitude          0
longtitude        0
city           2634
subclass1     60474
subclass2     60474
dtype: int64


In [5]:
pos_dataset.head()

,ID,latitude,longtitude,city,subclass1,subclass2
0,107780,22.297428,114.172607,overseas,公共机构/办公/住宅,公寓/小区/里弄
1,70990,31.136839,121.422639,shanghai,NaN,NaN
2,132379,31.232181,121.397632,shanghai,NaN,NaN
3,38132,34.266345,117.187751,xuzhou,餐饮,咖啡馆
4,104522,27.700000,85.333333,overseas,NaN,NaN


subclass1是位置的粗类别，Series中的value_counts函数可以将Series归类。粗类别中共有八类地点：

In [27]:
print('values in subclass 1:\n')
print(pos_dataset["subclass1"].value_counts())

values in subclass 1:

餐饮            33000
交通/住宿         14573
公共机构/办公/住宅    12162
商店/生活服务       11511
休闲/娱乐/文化       6271
景点/户外          5906
学校/教育机构        3623
其他地点           3586
Name: subclass1, dtype: int64


对细类别做同样的操作

In [28]:
print('values in subclass 2:\n')
pos_dataset["subclass2"].value_counts()

values in subclass 2:



其他            7267
公寓式酒店         5281
公寓/小区/里弄      4083
小吃            3341
火锅            3132
写字楼/办公室       3009
快餐            2859
咖啡馆           2616
面包/甜品         2126
公交站           1744
隐私地点          1568
日本料理          1550
四川菜           1500
医院/医疗中心/诊所    1481
西餐            1468
综合商场          1438
饮料/冰品         1425
时尚服饰          1376
道路/高速公路       1302
上海菜           1270
地铁站/轻轨站       1269
江浙菜           1259
广东菜           1174
便利店           1140
风景区           1059
机场            1026
酒吧             939
经济型酒店          932
公园             922
烧烤             892
              ... 
学校教室            52
停车场/车库          51
礼品店             46
药店              44
贵州菜             39
公司餐厅            37
烟/酒/茶商店         36
湖北菜             35
玩具店             33
软件/音像           32
学校实验室           29
旅行社             26
文具/办公用品         22
植物园             22
网吧              21
高尔夫球场           20
纹身店             18
澳门菜             15
花店              14
ATM             14
出租车停靠点          11
露营地         

### 读取用户信息

在内存较小的计算机中，在Jupyter中直接读取88M的源文件会导致程序崩溃。Pandas中可以指定每次读取文本的Chunksize从而避免内存问题，但是时间耗费还是很久。这里换了平常使用的windows机器才成功读取数据

In [29]:
train_datasrc = './data/train.txt'
train_dataset_chunk = pd.read_table(train_datasrc, header = None, delimiter = ',', names = ['UID', 'PID', 'visit_times'], iterator = True, chunksize = 10000)
train_dataset = pd.concat(train_dataset_chunk)

读取数据后，查看用户数据信息，发现数据比较规整，不包含NULL

In [9]:
train_dataset.info()
print('Columns with null values:\n', train_dataset.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043344 entries, 0 to 7043343
Data columns (total 3 columns):
UID            int64
PID            int64
visit_times    int64
dtypes: int64(3)
memory usage: 161.2 MB
Columns with null values:
 UID            0
PID            0
visit_times    0
dtype: int64


In [10]:
train_dataset.head()

,UID,PID,visit_times
0,0,0,1
1,0,1,1
2,0,2,1
3,0,3,1
4,0,4,1


## 数据清洗

### LabelEncoding

粗类别和细类别使用的是中文标签，

In [22]:
from sklearn.preprocessing import LabelEncoder
# astype('str')是必须的，因为待处理的Series里面包含了Nan值
# 参考https://stackoverflow.com/questions/46406720/labelencoder-typeerror-not-supported-between-instances-of-float-and-str
lb_make = LabelEncoder()
pos_dataset["subcode1"] = lb_make.fit_transform(pos_dataset["subclass1"].astype('str'))
pos_dataset[["subcode1", "subclass1"]].head(10)

,subcode1,subclass1
0,3,公共机构/办公/住宅
1,0,NaN
2,0,NaN
3,8,餐饮
4,0,NaN
5,1,交通/住宿
6,0,NaN
7,5,商店/生活服务
8,0,NaN
9,0,NaN


In [25]:
pos_dataset["subcode2"] = lb_make.fit_transform(pos_dataset["subclass2"].astype('str'))
pos_dataset[["subcode2", "subclass2"]].head(10)

,subcode2,subclass2
0,21,公寓/小区/里弄
1,3,NaN
2,3,NaN
3,39,咖啡馆
4,3,NaN
5,42,地铁站/轻轨站
6,3,NaN
7,84,时尚服饰
8,3,NaN
9,3,NaN


### One-Hot Encoding

### 相似度分析

皮尔逊相关系数

基于物品的相似度（item-based）和基于用户的相似度（user-based）

SVD

### 推荐系统的评价